# Equatorial twins
> Which world cities share the same absolute latitude? 

---

#### Import Python tools and Jupyter config

In [1]:
import os
import us
import json
import requests
import pandas as pd
import jupyter_black
import requests
import geopandas as gpd
import tempfile
from scipy.spatial import KDTree

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.display.max_colwidth = None

In [3]:
today = pd.Timestamp("today").strftime("%Y-%m-%d")

---

## Subhead, i.e. "Fetch"

#### Read data from XyXy source (live data where possible over local downloads)

In [4]:
# URL of the GeoJSON data
url = "https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/World_Cities/FeatureServer/0/query?outFields=*&where=1%3D1&f=geojson"

# Fetch the GeoJSON data using requests
response = requests.get(url)
response.raise_for_status()

# Create a temporary file to store the GeoJSON
with tempfile.NamedTemporaryFile(suffix=".geojson", delete=False) as tmp_file:
    tmp_file.write(response.content)  # Write the GeoJSON content to the file
    temp_file_path = tmp_file.name  # Get the file path

# Read the GeoJSON data into a GeoDataFrame
world_cities_gdf = gpd.read_file(temp_file_path)
world_cities_gdf.columns = world_cities_gdf.columns.str.lower()

# Clean up the temporary file
os.remove(temp_file_path)

In [5]:
capitals_gdf = world_cities_gdf.query('status == "National and provincial capital"')

In [6]:
# Function to mirror latitude
def mirror_latitude(row):
    return -row["latitude"]

In [7]:
world_cities_gdf.columns

Index(['fid', 'objectid', 'city_name', 'gmi_admin', 'admin_name', 'fips_cntry',
       'cntry_name', 'status', 'pop', 'pop_rank', 'pop_class', 'port_id',
       'label_flag', 'pop_source', 'geometry'],
      dtype='object')

In [8]:
# Add latitude and longitude columns
world_cities_gdf["latitude"] = world_cities_gdf.geometry.y.round(2)
world_cities_gdf["longitude"] = world_cities_gdf.geometry.x.round(2)

# Separate into Northern and Southern Hemisphere
northern_cities = world_cities_gdf[world_cities_gdf["latitude"] > 0].copy()
southern_cities = world_cities_gdf[world_cities_gdf["latitude"] < 0].copy()

# Mirror latitudes for the Southern Hemisphere
southern_cities["mirrored_latitude"] = southern_cities["latitude"].apply(
    lambda lat: -lat
)

# Create a KDTree for efficient nearest-neighbor search
northern_tree = KDTree(northern_cities[["latitude"]])

# Match Southern Hemisphere cities to their mirrored counterparts in the Northern Hemisphere
southern_cities["matched_city_index"] = northern_tree.query(
    southern_cities["mirrored_latitude"].values.reshape(-1, 1)
)[1]

# Add matched city and country information
southern_cities["matched_city"] = southern_cities["matched_city_index"].apply(
    lambda idx: northern_cities.iloc[idx]["city_name"]
)
southern_cities["matched_country"] = southern_cities["matched_city_index"].apply(
    lambda idx: northern_cities.iloc[idx]["cntry_name"]
)

# Display the Southern Hemisphere cities with their Northern Hemisphere matches
equitorial_twins_gdf = southern_cities[
    [
        "city_name",
        "cntry_name",
        "latitude",
        "mirrored_latitude",
        "matched_city",
        "matched_country",
    ]
].reset_index(drop=True)

In [9]:
equitorial_twins_gdf

,city_name,cntry_name,latitude,mirrored_latitude,matched_city,matched_country
0,Cuiaba,Brazil,-15.62,15.62,Omdurman,Sudan
1,Brasilia,Brazil,-15.79,15.79,Yasothon,Thailand
2,Goiania,Brazil,-16.73,16.73,Tuxtla Gutierrez,Mexico
3,Campo Grande,Brazil,-20.45,20.45,Thai Binh,Vietnam
4,Pedro Juan Caballero,Paraguay,-22.53,22.53,Kolkata,India
...,...,...,...,...,...,...
208,Wanganui,New Zealand,-39.93,39.93,Ankara,Turkey
209,Wellington,New Zealand,-41.33,41.33,Durres,Albania
210,Rockhampton,Australia,-23.43,23.43,Mazatlan,Mexico
211,Brisbane,Australia,-27.45,27.45,Thimphu,Bhutan
